MAIN LINK -> https://github.com/afzalsayed96/TwitterLive

https://github.com/sarthak0797/Live-Twitter-Sentiment-Analysis (isme main scraper file se kam hjaega shyd)

https://github.com/varunu28/Twitter-HashTag-Streamer

https://github.com/sonal-sannigrahi/TwitterLive (siple and small yet)

In [1]:

import tweepy,json
#access_token=""
csecret="8njgmX86sIfVquvyxjmtWrm95iK9ugS1tbOPTNtyCkP6eHpQME"
ckey="mUUffXtBjTu73oIAHtxJxGdpI"

# callback_uri = 'oob' # https://cfe.sh/twitter/callback


# auth = tweepy.OAuthHandler(ckey, csecret, callback_uri)
# redirect_url = auth.get_authorization_url()
# print(redirect_url)

In [2]:
# import webbrowser
# webbrowser.open(redirect_url)

# user_pint_input = input("What's the pin value? ")

# user_pint_input

# auth.get_access_token(user_pint_input)

# print(auth.access_token, auth.access_token_secret)

In [11]:
# atoken=auth.access_token
# asecret=auth.access_token_secret
atoken = "1675996254-lAEltpcg4EkTFNQECA2ytxCjKiR1sLdGy7g8XAV"
asecret = "x4ZA9YaeUb5qTmh3BYAPaInmgVqNDybqwcODM5BNazeED"
print(atoken) # 1675996254-lAEltpcg4EkTFNQECA2ytxCjKiR1sLdGy7g8XAV
print(asecret) # x4ZA9YaeUb5qTmh3BYAPaInmgVqNDybqwcODM5BNazeED


1675996254-lAEltpcg4EkTFNQECA2ytxCjKiR1sLdGy7g8XAV
x4ZA9YaeUb5qTmh3BYAPaInmgVqNDybqwcODM5BNazeED


In [16]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
# importing the csv module
import csv
import datetime
import json
import os
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# analyser = SentimentIntensityAnalyzer()

#consumer key, consumer secret, access token, access secret.
# ckey=""
# csecret=""
# atoken="-"
# asecret=""


# current_time = datetime.datetime.now().strftime("%H:%M:%S")
#     current_time = datetime.datetime.utcnow().strftime("%H:%M")


start_time = datetime.datetime.now()

class listener(StreamListener):

    def on_data(self, data):
        global start_time
        
        all_data = json.loads(data)
        print(len(all_data))
        Elon_Musk_Flag = []
        tweet = all_data["text"]
        
        username = all_data["user"]["screen_name"]
        if(username == 'elonmusk'):
            Elon_Musk_Flag.append(1)
        else:
            Elon_Musk_Flag.append(0)
#         print(username, Elon_Musk_Flag)
        date = all_data["created_at"]

        followers = all_data["user"]["followers_count"]
        
        if all_data["user"]["verified"]:
            verified = 'Yes'
        else:
            verified = 'No'

        retweet = all_data["retweet_count"]

        name = all_data["user"]["name"]
                            
        tweet_url = 'https://twitter.com/'+all_data["user"]["screen_name"]+'/status/'+str(all_data["id"])
#         fields = ['Date','Name', 'Username', 'Tweet', 'Followers', 'Verified', 'URL', 'Retweets']
        
        fields_sheet_1 = ['Date','Tweet', 'Followers','Elon_Musk_Flag' 'URL']
        fields_sheet_2 = ['Date','Tweet', 'Followers','Elon_Musk_Flag' 'URL']
        
        # data rows of csv file
#         rows = [ [date, name, username, tweet, followers, verified,tweet_url, retweet]]
        rows_sheet_1 = [ [date, tweet, followers, Elon_Musk_Flag, tweet_url]]
        rows_sheet_2 = [ [date, tweet, followers, Elon_Musk_Flag, tweet_url]]
        
        # name of csv file
        filename = datetime.datetime.now().strftime("%Y-%m-%d")+".csv"
    
        print("Current Time =", datetime.datetime.now().strftime("%H:%M:%S"))
        current_time = datetime.datetime.now()
        
        difference = current_time - start_time
        
        if difference.total_seconds() > 60:
            print('1 Minute Elapsed')
            start_time = current_time
            
        # writing to csv file
        with open(filename, 'a+',encoding="utf-8") as csvfile:
            # creating a csv writer object
            csvwriter = csv.writer(csvfile)
            print('Data Written')
            
            # writing the fields
            if not(os.path.exists(filename) and os.path.getsize(filename) > 0):
                csvwriter.writerow(fields_sheet_1)
            
            # writing the data rows
            csvwriter.writerows(rows_sheet_1)

#         print((date,username,tweet, followers,Elon_Musk_Flag))
#         print(len(all_data))
        insights()
        return True

    def on_error(self, status):
        print (status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
#Enter Your keywords here
keywords = ["python"]

import time
i=0
while(True):
    
    twitterStream = Stream(auth, listener())
    twitterStream.filter(track=keywords)

28
Current Time = 14:49:02
Data Written
28
Current Time = 14:49:03
Data Written
28
Current Time = 14:49:09
Data Written
29
Current Time = 14:49:10
Data Written
27
Current Time = 14:49:11
Data Written
29
Current Time = 14:49:12
Data Written
29
Current Time = 14:49:13
Data Written
29
Current Time = 14:49:16
Data Written
30
Current Time = 14:49:16
Data Written
28
Current Time = 14:49:19
Data Written
27
Current Time = 14:49:20
Data Written
27
Current Time = 14:49:21
Data Written
28
Current Time = 14:49:21
Data Written
28
Current Time = 14:49:21
Data Written
28
Current Time = 14:49:23
Data Written
28
Current Time = 14:49:23
Data Written
29
Current Time = 14:49:24
Data Written
29
Current Time = 14:49:27
Data Written
28
Current Time = 14:49:27
Data Written
28
Current Time = 14:49:28
Data Written
29
Current Time = 14:49:29
Data Written
28
Current Time = 14:49:29
Data Written
28
Current Time = 14:49:30
Data Written
30
Current Time = 14:49:35
Data Written
29
Current Time = 14:49:35
Data Written


KeyboardInterrupt: 